In [3]:
import os;
import sys;
import pickle;
import numpy as np;

In [4]:
def load_CIFAR10(ROOT):
    """
    load entire CIFAR-10 dataset
    
    code is adapted from CS231n assignment kit
    
    @param ROOT: string of data folder
    @return: Xtr, Ytr: training data and labels
    @return: Xte, Yte: testing data and labels
    """
    
    xs=[];
    ys=[];
    
    for b in range(1,6):
        f=os.path.join(ROOT, "data_batch_%d" % (b, ));
        X, Y=load_CIFAR_batch(f);
        xs.append(X);
        ys.append(Y);
        

    Xtr=np.concatenate(xs);
    Ytr=np.concatenate(ys);
    del X, Y;
    
    Xte, Yte=load_CIFAR_batch(os.path.join(ROOT, "test_batch"));
    
    return Xtr, Ytr, Xte, Yte;

In [5]:
def load_CIFAR_batch(filename):
    """
    load single batch of cifar-10 dataset
    
    code is adapted from CS231n assignment kit
    
    @param filename: string of file name in cifar
    @return: X, Y: data and labels of images in the cifar batch
    """
    
    with open(filename, 'rb') as f:
        if sys.version_info.major == 2:
            datadict = pickle.load(f)
        elif sys.version_info.major == 3:
            datadict = pickle.load(f, encoding='latin-1')
        
        X=datadict['data'];
        Y=datadict['labels'];
        
        X=X.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("float");
        Y=np.array(Y);
        
        return X, Y;
    

In [15]:
def L_i(x, y, W):
    """
    unvectorized version. Compute the multiclass svm loss for a single example (x,y)
    - x is a column vector representing an image (e.g. 3073 x 1 in CIFAR-10)
    with an appended bias dimension in the 3073-rd position (i.e. bias trick)
    - y is an integer giving index of correct class (e.g. between 0 and 9 in CIFAR-10)
    - W is the weight matrix (e.g. 10 x 3073 in CIFAR-10)
    """
    delta = 1.0 # see notes about delta later in this section
    scores = W.dot(x) # scores becomes of size 10 x 1, the scores for each class
    correct_class_score = scores[y]
    D = W.shape[0] # number of classes, e.g. 10
    loss_i = 0.0
    for j in xrange(D): # iterate over all wrong classes
        if j == y:
            # skip for the true class to only loop over incorrect classes
            continue
        # accumulate loss for the i-th example
        loss_i += max(0, scores[j] - correct_class_score + delta)
    return loss_i

def L_i_vectorized(x, y, W):
    """
    A faster half-vectorized implementation. half-vectorized
    refers to the fact that for a single example the implementation contains
    no for loops, but there is still one loop over the examples (outside this function)
    """
    delta = 1.0
    scores = W.dot(x)
    # compute the margins for all classes in one vector operation
    margins = np.maximum(0, scores - scores[y] + delta)
    # on y-th position scores[y] - scores[y] canceled and gave delta. We want
    # to ignore the y-th position and only consider margin on max wrong class
    margins[y] = 0
    loss_i = np.sum(margins)
    return loss_i

def L(X, y, W):
    """ 
    fully-vectorized implementation :
    - X holds all the training examples as columns (e.g. 3073 x 50,000 in CIFAR-10)
    - y is array of integers specifying correct class (e.g. 50,000-D array)
    - W are weights (e.g. 10 x 3073)
    """
    # evaluate loss over all examples in X without using any for loops
    # left as exercise to reader in the assignment
    delta = 1.0
    scores = W.dot(X)
    num = scores.shape[1]
    # remember interger indexing in python, very important(different from MATLAB)
    margins = np.maximum(0, scores - scores[y,np.arange(num)] + delta); 
    margins[y,np.arange(num)] = 0
    loss_i = np.sum(margins)/scores.shape[1]
    return loss_i
def regulizaion(W):
    londa = 1.0
    rW=np.sum(np.square(W,W))
    return rW

In [16]:
Xtr, Ytr, Xte, Yte = load_CIFAR10('cifar10/') # a magic function we provide
# flatten out all images to be one-dimensional
Xtr_rows = Xtr.reshape(Xtr.shape[0], 32 * 32 * 3) # Xtr_rows becomes 50000 x 3072
Xte_rows = Xte.reshape(Xte.shape[0], 32 * 32 * 3) # Xte_rows becomes 10000 x 3072
Xtr_cols = np.concatenate((np.ones((1,50000)),np.transpose(Xtr_rows)))
Xte_cols = np.concatenate((np.ones((1,10000)),np.transpose(Xte_rows)))

In [19]:
def result(W):
    # Assume X_test is [3073 x 10000], Y_test [10000 x 1]
    scores = W.dot(Xte_cols) # 10 x 10000, the class scores for all test examples
    # find the index with max score in each column (the predicted class)
    Yte_predict = np.argmax(scores, axis = 0)
    # and calculate accuracy (fraction of predictions that are correct)
    print ('accuracy: %f' % ( np.mean((Yte_predict == Yte).astype(np.float32)) ))

# Strategy 1: Random search

In [8]:
# assume X_train is the data where each column is an example (e.g. 3073 x 50,000)
# assume Y_train are the labels (e.g. 1D array of 50,000)
# assume the function L evaluates the loss function

bestloss = float("inf") # Python assigns the highest possible float value
for num in range(1000):
    W = np.random.randn(10, 3073) * 0.0001 # generate random parameters
    loss = L(Xtr_cols, Ytr, W) # get the loss over the entire training set
    if loss < bestloss: # keep track of the best solution
        bestloss = loss
        Wbest = W
    print ('in attempt %d the loss was %f, best %f' % (num, loss, bestloss))

result(Wbest)

in attempt 0 the loss was 10.252153, best 10.252153
in attempt 1 the loss was 9.945788, best 9.945788
in attempt 2 the loss was 9.526387, best 9.526387
in attempt 3 the loss was 10.537743, best 9.526387
in attempt 4 the loss was 10.298508, best 9.526387
in attempt 5 the loss was 9.897546, best 9.526387
in attempt 6 the loss was 9.260174, best 9.260174
in attempt 7 the loss was 9.562320, best 9.260174
in attempt 8 the loss was 9.623974, best 9.260174
in attempt 9 the loss was 10.856661, best 9.260174
in attempt 10 the loss was 10.260102, best 9.260174
in attempt 11 the loss was 10.324253, best 9.260174
in attempt 12 the loss was 9.481508, best 9.260174
in attempt 13 the loss was 9.650705, best 9.260174
in attempt 14 the loss was 9.852696, best 9.260174
in attempt 15 the loss was 9.849692, best 9.260174
in attempt 16 the loss was 9.844130, best 9.260174
in attempt 17 the loss was 10.519446, best 9.260174
in attempt 18 the loss was 9.493455, best 9.260174
in attempt 19 the loss was 9.7960

0.12429999999999999

# Strategy 2: Random Local Search

In [20]:
W = np.random.randn(10, 3073) * 0.001 # generate random starting W
bestloss = float("inf")
for i in range(1000):
    step_size = 0.0001
    Wtry = W + np.random.randn(10, 3073) * step_size
    loss = L(Xtr_cols, Ytr, Wtry)
    if loss < bestloss:
        W = Wtry
        bestloss = loss
    print ('iter %d loss is %f' % (i, bestloss))
result(W)

iter 0 loss is 45.345350
iter 1 loss is 44.377905
iter 2 loss is 44.377905
iter 3 loss is 44.048173
iter 4 loss is 42.654332
iter 5 loss is 42.654332
iter 6 loss is 42.654332
iter 7 loss is 42.654332
iter 8 loss is 42.348042
iter 9 loss is 42.348042
iter 10 loss is 41.412049
iter 11 loss is 41.412049
iter 12 loss is 41.412049
iter 13 loss is 41.315959
iter 14 loss is 41.315959
iter 15 loss is 41.156427
iter 16 loss is 40.875113
iter 17 loss is 40.875113
iter 18 loss is 40.065205
iter 19 loss is 39.379616
iter 20 loss is 39.379616
iter 21 loss is 39.379616
iter 22 loss is 39.207450
iter 23 loss is 39.207450
iter 24 loss is 39.207450
iter 25 loss is 39.207450
iter 26 loss is 38.401821
iter 27 loss is 38.401821
iter 28 loss is 38.401821
iter 29 loss is 37.537244
iter 30 loss is 36.744847
iter 31 loss is 36.744847
iter 32 loss is 36.744847
iter 33 loss is 35.582626
iter 34 loss is 35.582626
iter 35 loss is 34.292754
iter 36 loss is 33.643504
iter 37 loss is 33.643504
iter 38 loss is 33.643

# Strategy 3: Following the Gradient

In [25]:
def eval_numerical_gradient(f, x):
    """ 
    a naive implementation of numerical gradient of f at x 
    - f should be a function that takes a single argument
    - x is the point (numpy array) to evaluate the gradient at
    """ 

    fx = f(x) # evaluate function value at original point
    grad = np.zeros(x.shape)
    h = 0.00001

    # iterate over all indexes in x
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:

        # evaluate function at x+h
        ix = it.multi_index
        old_value = x[ix]
        x[ix] = old_value + h # increment by h
        fxh = f(x) # evalute f(x + h)
        x[ix] = old_value # restore to previous value (very important!)

        # compute the partial derivative
        grad[ix] = (fxh - fx) / h # the slope
        it.iternext() # step to next dimension

    return grad

In [29]:
# to use the generic code above we want a function that takes a single argument
# (the weights in our case) so we close over X_train and Y_train
def CIFAR10_loss_fun(W):
    return L(Xtr_cols, Ytr, W)

W = np.random.rand(10, 3073) * 0.001 # random weight vector
df = eval_numerical_gradient(CIFAR10_loss_fun, W) # get the gradient
loss_original = CIFAR10_loss_fun(W) # the original loss
print ('original loss: %f' % (loss_original, ))

# lets see the effect of multiple step sizes
for step_size_log in [10, 9, 8, 7, 6, 5, 4, 3, 2, 1]:
    step_size = 10 ** step_size_log
    W_new = W - step_size * df # new position in the weight space
    loss_new = CIFAR10_loss_fun(W_new)
    print ('for step size %f new loss: %f' % (step_size, loss_new))

KeyboardInterrupt: 

In [ ]:
# Vanilla Minibatch Gradient Descent

while True:
    data_batch = sample_training_data(data, 256) # sample 256 examples
    weights_grad = evaluate_gradient(loss_fun, data_batch, weights)
    weights += - step_size * weights_grad # perform parameter update